### 작업형 제1유형

#### 1.

In [8]:
import pandas as pd

df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/501_trash_bag.csv', encoding='cp949')
new_df = df.loc[(df['용도'] == '음식물쓰레기') & (df['사용대상'] == '가정용')]

print(int(new_df.loc[new_df['2L가격'] != 0, '2L가격'].mean()))

119


#### 2.

In [15]:
df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/502_bmi.csv')
df['BMI'] = df['Weight'] / ((df['Height'] / 100) ** 2)

normal = len(df.loc[(df['BMI'] >= 18.5) & (df['BMI'] < 23), :])
non_normal = len(df.loc[(df['BMI'] >= 23) & (df['BMI'] < 25), :])

print(abs(normal - non_normal))

28


#### 3.

In [25]:
df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/503_students.csv', encoding='cp949')

# 순 전입학생수 = 총 전입학생수 - 총 전출학생수
df['순 전입학생'] = df['총 전입학생'] - df['총 전출학생']

# df.groupby('학교')['순 전입학생'].sum()

print(df[df['학교'] == 'A']['전체 학생 수'].sum())

566


<br><br><br>

### 작업형 제2유형

In [64]:
import pandas as pd
import numpy as np

X_train = pd.read_csv('../yemoonsaBigdata/datasets/Part3/504_x_train.csv')
y_train = pd.read_csv('../yemoonsaBigdata/datasets/Part3/504_y_train.csv')
X_test = pd.read_csv('../yemoonsaBigdata/datasets/Part3/504_x_test.csv')


remove_columns = ['id']
num_columns = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
cat_columns = ['model', 'transmission', 'fuelType']
columns_y = ['price']

# 범주형 데이터 전처리
from sklearn.preprocessing import LabelEncoder

X = pd.concat([X_train, X_test])

for col in cat_columns:
    label_encoder = LabelEncoder()
    
    label_encoder.fit(X[col])
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])    
    

# 학습, 검증 데이터셋으로 분리
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train[num_columns+cat_columns], y_train[columns_y], test_size=0.3)


# 수치형 데이터 전처리
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[num_columns])

X_tr[num_columns] = scaler.transform(X_tr[num_columns])
X_val[num_columns] = scaler.transform(X_val[num_columns])
X_test[num_columns] = scaler.transform(X_test[num_columns])



# 학습
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

'''
model_rf = RandomForestRegressor()
model_rf.fit(X_tr, y_tr)

model_xgb = XGBRegressor()
model_xgb.fit(X_tr, y_tr)

from sklearn.metrics import mean_squared_error

pred_rf = model_rf.predict(X_val)
rmse_rf = mean_squared_error(y_val, pred_rf) ** 0.5
print(rmse_rf)

pred_rf = model_xgb.predict(X_val)
rmse_rf = mean_squared_error(y_val, pred_rf) ** 0.5
print(rmse_rf)
'''


# 하이퍼 파라미터
from sklearn.metrics import mean_squared_error

# model_xgb1 = XGBRegressor(n_estimators=50, max_depth=5)
# model_xgb1.fit(X_tr[num_columns + cat_columns], y_tr)
# pred_xgb1 = model_xgb1.predict(X_val)
# rmse_xgb1 = mean_squared_error(y_val, pred_xgb1) ** 0.5
# print(rmse_xgb1)


# model_xgb2 = XGBRegressor(n_estimators=50, max_depth=10)
# model_xgb2.fit(X_tr, y_tr)
# pred_xgb2 = model_xgb2.predict(X_val)
# rmse_xgb2 = mean_squared_error(y_val, pred_xgb2) ** 0.5
# print(rmse_xgb2)


model_xgb3 = XGBRegressor(n_estimators=100, max_depth=5)
model_xgb3.fit(X_tr, y_tr)
pred_xgb3 = model_xgb3.predict(X_val)
rmse_xgb3 = mean_squared_error(y_val, pred_xgb3) ** 0.5
print(rmse_xgb3)


# model_xgb4 = XGBRegressor(n_estimators=100, max_depth=10)
# model_xgb4.fit(X_tr, y_tr)
# pred_xgb4 = model_xgb4.predict(X_val)
# rmse_xgb4 = mean_squared_error(y_val, pred_xgb4) ** 0.5
# print(rmse_xgb4)


pred = model_xgb3.predict(X_test[num_columns+cat_columns])
result = pd.DataFrame({
    'id': X_test['id'],
    'price': pred
})

result.to_csv('../yemoonsaBigdata/res/1.csv', index=False)


2467.1320201362946
